In [212]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import pandas as pd
import plotly.express as px
from scipy.fft import fft,ifft
import matplotlib.pyplot as plt
import os

In [213]:
#getting experiment list
experiments = os.listdir("Person_A")
for i in range(len(experiments)):
    experiments[i] = experiments[i][:-4]
experiments.sort()

#loading data
def load(person = "C",exp="Reference"):
    return pd.read_csv("Person_"+person+"/"+exp+".csv",header=None,names=["vals"])

#rectify signal
def rectify(ref,data):
    data.vals = np.absolute(data.vals)-ref.vals.mean()
    return data

def fft_filter(ref,data):
    num = max(len(data.vals),len(ref.vals))
    data.vals = ifft(fft(np.array(data.vals).real,n=num)-fft(np.array(ref.vals).real,n=num))
    return data
def moving_average(data,window_size):
    data.vals = data.vals.rolling(window_size).mean()
    return data
def plot(data,stdeb=False):
    plt.plot(data)
    plt.show()

In [214]:
def graph(person,exp,rc=True,ffilt=True,ma=True,stdeb=True):
    if(exp == "Reference"): ffilt = False #if looking at reference signal, can't filter by itself
    ref = load(person,"Reference") #reference signal for person
    sig = load(person,exp) #signal of interest
    if(rc): sig = rectify(ref,sig)
    if(ffilt): sig = fft_filter(ref,sig)
    if(ma!=0): sig = moving_average(sig,ma)
    plot(sig,stdeb)

In [215]:
ref = load("A")
sig = load("A","flex_fingers1")
print(len(ref))
print(len(sig))
interact(graph, person=["A","B","C"],exp=experiments,rc=True,ffilt=False,ma=75,stdeb=True);

2500
5000


interactive(children=(Dropdown(description='person', options=('A', 'B', 'C'), value='A'), Dropdown(description…